In [ ]:
import hopsworks
import pandas as pd
import tensorflow as tf
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential
import os
import numpy as np
import joblib
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# # You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
# Get features (data) from feature store
# nasdaq data
nasdaq_fg = fs.get_feature_group(name="nasdaq", version=1)
print(nasdaq_fg)
query = nasdaq_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="nasdaq",
                                  version=1,
                                  description="Read from nasdaq dataset",
                                  labels=["close"],
                                  query=query)
print(feature_view)

df_nasdaq = nasdaq_fg.read()
df_nasdaq = df_nasdaq.sort_values(by="date")
df_nasdaq.reset_index(drop=True,inplace=True)

print(df_nasdaq)
y_nasdaq = df_nasdaq['close'].values.reshape(-1,1)

In [ ]:
# forecast setting
n_forecast = 90  # length of output sequences (forecast period)
n_lookback = 3*n_forecast  # length of input sequences (lookback period, should be 3 times longer than forecast period)

In [ ]:
# Model Training for nasdaq
scaler_nasdaq = MinMaxScaler(feature_range=(0, 1))
scaler_nasdaq = scaler_nasdaq.fit(y_nasdaq)
y_nasdaq = scaler_nasdaq.transform(y_nasdaq)

X_nasdaq = []
Y_nasdaq = []

for i in range(n_lookback, len(y_nasdaq) - n_forecast + 1):
    X_nasdaq.append(y_nasdaq[i - n_lookback: i])
    Y_nasdaq.append(y_nasdaq[i: i + n_forecast])

X_nasdaq = np.array(X_nasdaq)
Y_nasdaq = np.array(Y_nasdaq)
print(X_nasdaq.shape, Y_nasdaq.shape)

# fit / train the model
model_nasdaq = Sequential()
model_nasdaq.add(LSTM(units=128, return_sequences=True, input_shape=(n_lookback, 1)))
model_nasdaq.add(LSTM(units=64, return_sequences=True))
model_nasdaq.add(LSTM(units=64, return_sequences=False))
model_nasdaq.add(Dense(n_forecast))
model_nasdaq.compile(loss='mean_squared_error', optimizer='adam')
model_nasdaq.fit(X_nasdaq, Y_nasdaq, epochs=20, batch_size=32, verbose=1)

In [ ]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()
model_dir="nasdaq_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model_nasdaq, model_dir + "/nasdaq_model.pkl")
# fig.savefig(model_dir + "/wine_confusion_matrix.png")

# # Create an entry in the model registry that includes the model's name, desc, metrics
nasdaq_model = mr.python.create_model(
    name="nasdaq_model",
    # metrics={"accuracy" : metrics['accuracy']},
    # model_schema=model_schema,
    description="nasdaq Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
nasdaq_model.save(model_dir)
